In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from pprint import pprint

load_dotenv("../.env")

client = OpenAI(api_key=os.environ['API_KEY'])

MODEL_NAME = "gpt-4o-mini"

In [2]:
import json

# 検索結果を返す
def get_search_result(keyword, when="1", unit="m"):
    result = {
        "result": [
            {"title": "1914年（大正3年）中央停車場改め「東京駅」として営業を開始しました"},
            {"title": "1929年（昭和4年）八重洲口が開設されました"},
            {"title": "2014年（平成26年）東京駅開業100周年を迎えました"},  
        ]
    }

    return json.dumps(result)

In [3]:
from openai.types.chat import ChatCompletionToolParam

# ツール定義
tools=[
    ChatCompletionToolParam({
        "type": "function",
        "function": {
            "name": "get_search_result",
            "description": "指定したキーワードの検索結果を取得する",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "キーワード",
                    },
                    "when": {
                        "type": "number",
                        "description": "日付や時間の範囲"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["d", "h", "y"]
                    },
                },
                "required": ["keyword"],
            },
        },
    })
]

In [4]:
question = "日本の首都はどこにありますか？"

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": question},
    ],
    tools=tools,
    tool_choice="auto",
)

pprint(vars(response))

{'_request_id': 'req_c5ce738d551f4e8caaf816f9cb63434e',
 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='日本の首都は東京にあります。東京は日本の政治、経済、文化の中心地であり、特に東京23区がその中心となっています。', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))],
 'created': 1756691909,
 'id': 'chatcmpl-CAo6PvrYU1UjLL07XLNiuaXOnp552',
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_8bda4d3a2c',
 'usage': CompletionUsage(completion_tokens=37, prompt_tokens=91, total_tokens=128, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))}


In [5]:
# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": question},
    ],
    tools=tools,
    tool_choice="auto",
)

# レスポンスの表示
pprint(vars(response))


{'_request_id': 'req_4ff26313279e47238c0f82ffcebf392e',
 'choices': [Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_2gRdReZJDoplOwVptF2FwxDg', function=Function(arguments='{"keyword":"東京駅 イベント","when":30,"unit":"d"}', name='get_search_result'), type='function')]))],
 'created': 1756691976,
 'id': 'chatcmpl-CAo7UEqfcmqYWwfFtkJew5wjqq3A4',
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_8bda4d3a2c',
 'usage': CompletionUsage(completion_tokens=26, prompt_tokens=97, total_tokens=123, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))}


In [6]:
if response.choices[0].finish_reason == 'tool_calls':
    # 言語モデルの回答からツールを取得
    tool = response.choices[0].message.tool_calls[0]

    # 関数名の取得
    function_name = tool.function.name
    print(f"関数名：{function_name}")

    # 引数取得
    arguments = json.loads(tool.function.arguments)
    print(f"引数：{arguments}")

    # 関数実行
    function_response = globals()[function_name](**arguments)
    print("関数の実行結果：")
    pprint(json.loads(function_response))

    # 関数の実行結果をmessagesに加えて言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )

    # 言語モデルからの回答を出力
    print("言語モデルからの回答：")
    print(response_after_tool_call.choices[0].message.content.strip())

else:
    # 関数呼び出しでなければ、単に言語モデルからの回答を出力
    print("言語モデルからの回答：")
    print(response.choices[0].message.content.strip())

関数名：get_search_result
引数：{'keyword': '東京駅 イベント', 'when': 30, 'unit': 'd'}
関数の実行結果：
{'result': [{'title': '1914年（大正3年）中央停車場改め「東京駅」として営業を開始しました'},
            {'title': '1929年（昭和4年）八重洲口が開設されました'},
            {'title': '2014年（平成26年）東京駅開業100周年を迎えました'}]}
言語モデルからの回答：
最近1ヶ月以内に東京駅で行われたイベントの情報は以下の通りです。

1. **1914年（大正3年）中部停車場改名「東京駅」として業務を開始しました**
   - 東京駅が1914年に業務を開始したことを記念するイベント。

2. **1929年（昭和4年）八重洲口が開設されました**
   - 東京駅の歴史に関する展示やイベントが行われた模様。

3. **2014年（平成26年）東京駅開業100周年を迎えました**
   - 東京駅開業100周年を記念したイベントや展示が開催されていた。

これらのイベントは、東京駅の歴史や地域の発展に関連する内容で、多くの来場者が訪れたと考えられます。詳細な日程や内容については、東京駅の公式ウェブサイトや関連する発表を参照することをおすすめします。
